# Initialize imports

In [1]:
from classes import *
from functions import *
import pandas as pd
import numpy as np

# Setting up Y_Bus, bus and line objects for all task

In [2]:
line_data = ReadCsvFile('./files/network_configuration_line_data.csv')
bus_data = ReadCsvFile('./files/network_configuration_bus_data.csv')

Sbase = 100 # MVA
Ubase = 230 # kV
num_buses = len(bus_data)
YBus = BuildYbusMatrix(line_data, num_buses)
BusList = buildBusList(bus_data, Sbase)
bus_overview = setupBusType(bus_data)


# Newton Raphson Load Flow Algorithm

In [3]:
P_spec, Q_spec = findKnowns(bus_data, Sbase)
v_guess, dirac_guess = findUnknowns(bus_overview, bus_data)

jacobian_matrix = buildJacobian(BusList, P_spec, Q_spec, v_guess, dirac_guess, YBus)

deltaP = calcP(BusList, P_spec, YBus, Sbase)
deltaQ = calcQ(BusList, Q_spec, YBus, Sbase)
knowns = np.concatenate((deltaP, deltaQ), axis= 0)

# df_inv_jac = pd.DataFrame(np.linalg.pinv(df_jac))
# print(df_inv_jac)
unknowns = calcDeltaUnknowns(jacobian_matrix, knowns)
print(unknowns)

updateVoltageAndAngleList(unknowns, dirac_guess, v_guess)
updateBusList(BusList, dirac_guess, v_guess)

runNewtonRaphson()


[[ 0.0314373 ]
 [ 0.00898817]
 [-0.0162399 ]
 [-0.02418557]
 [-0.04669507]
 [-0.02927996]
 [-0.0090333 ]]


# Decoupled Load Flow

In [ ]:
dlf_jacobian = calcDecoupledJacobian(BusList, P_spec, Q_spec, v_guess, dirac_guess, YBus)
print(dlf_jacobian)
dlf_unknowns= calcDecoupledDiracVoltage(dlf_jacobian, knowns)
print(dlf_unknowns)

# Fast Decoupled Load Flow

# DC Power Flow